In [1]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import numpy as np
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import re

# Clean and tokenize text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer


2023-05-10 12:37:18.434516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hannah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Hannah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [51]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [56]:
df = pd.read_csv('transformers_dataNew.csv')

In [57]:
df.dropna(subset=['content'], inplace=True)

In [58]:
df

,name,shares,wow,cares,sad,angry,haha,reactions_count,comments,content,...,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767,embeddings
0,RTL Nieuws,0,17,0,0,0,14,39,0,Medewerkers van een aquarium in Spanje aten ge...,...,13.776934,0.816872,0.773461,1.520865,1.432767,1.400064,0.703925,0.769034,1.060223,[ 2.12260270e+00 1.95452666e+00 6.88976407e-...
1,RTL Nieuws,0,1,0,0,0,10,15,0,"""Er wordt wiet gerookt in het boek, gezoend, e...",...,13.530748,1.590019,1.094324,1.871268,1.652691,1.231773,1.189072,1.068386,1.233858,[ 2.13255215e+00 1.18901312e+00 7.16963530e-...
2,RTL Nieuws,1,0,11,0,0,38,67,206,"Andy en het team van de dragbar laten zich, on...",...,13.372700,1.080609,0.964037,1.063937,1.858110,1.554261,0.934504,0.737435,1.404058,[ 2.12687016e+00 1.41113973e+00 9.98041630e-...
3,RTL Nieuws,2,4,7,19,0,0,30,26,Onder de slachtoffers waren volgens VN-mensenr...,...,13.289650,1.249961,1.254739,1.537239,1.924307,1.640161,0.366889,0.969132,0.716310,[ 2.9026854 0.780339 1.11986625 1.063458...
4,RTL Nieuws,5,12,0,0,0,21,58,284,Alleen al de afgelopen drie dagen kwamen meer ...,...,13.318510,1.263664,0.760335,1.193509,1.659033,1.697025,1.157894,1.496184,1.604272,[ 1.75874364 0.47566167 1.14991474 0.760251...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,De Telegraaf,3,0,0,0,0,47,103,169,Elke aardbewoner surft jaarlijks gemiddeld mee...,...,13.060578,1.298744,0.832913,1.100882,1.712117,0.040602,0.658332,1.102006,1.012051,[ 2.00863171 1.04274106 0.94798875 0.684139...
9996,De Telegraaf,4,0,0,0,0,17,60,26,De Nederlander en zijn team Red Bull hadden de...,...,13.239983,0.822294,0.695602,1.593539,1.619219,1.291816,0.724189,0.918574,1.635511,[ 1.101861 1.73083413 0.7798183 0.473033...
9997,De Telegraaf,2,0,0,0,23,39,71,65,„We weten weinig van de mogelijke effecten van...,...,12.788166,1.192427,0.812722,1.040017,1.249091,1.165109,0.630857,1.169717,1.338238,[ 2.35046268 1.5198431 0.71057779 1.275846...
9998,De Telegraaf,3,0,0,0,0,60,91,135,Het rommelt binnen het CDA. De Telegraaf,...,12.757520,0.974428,0.540617,1.341013,1.987473,0.697544,0.471119,0.873903,1.283651,[ 2.13793945e+00 1.02813029e+00 3.28069597e-...


In [63]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the data
df = pd.read_csv('transformers_dataNew.csv')
#df = df[df['reactions_count'] > 30]

# Split the data into train, validation, and test sets
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")

# Tokenize and encode the text data
train_tokens = tokenizer.batch_encode_plus(
    train_data['content'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

valid_tokens = tokenizer.batch_encode_plus(
    valid_data['content'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

test_tokens = tokenizer.batch_encode_plus(
    test_data['content'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

# Convert the tokenized input to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    train_data['entropy'].values
))

valid_dataset = tf.data.Dataset.from_tensor_slices((
    dict(valid_tokens),
    valid_data['entropy'].values
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    test_data['entropy'].values
))

# Prepare the model input
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# Load the BERT model
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow

# Freeze the BERT layers
model.trainable = False

# Retrieve the BERT embeddings
embeddings = model(input_ids, attention_mask=attention_mask)[0]

# Perform pooling (average pooling in this case)
pooled_output = tf.keras.layers.GlobalAveragePooling1D()(embeddings)

# Add a dense layer for regression
output = tf.keras.layers.Dense(1, activation='linear')(pooled_output)

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mean_squared_error')

# Train the model
model.fit(
    train_dataset.batch(32),
    epochs=5,
    validation_data=valid_dataset.batch(32)
)

# Evaluate the model on the test set
predictions = model.predict(test_dataset.batch(32))
mse = mean_squared_error(test_data['entropy'].values, predictions)
print('Mean Squared Error:', mse)


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

## From https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794, Language model approach
